<a href="https://colab.research.google.com/github/pandian-raja/EVA8/blob/feature%2Fs3/S_3_Backpropagation_Architectural_Basics/Part3/EVA4_Session_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.sequ1 = nn.Sequential(
            nn.Conv2d(1, 8, 3), #input -28 OUtput - 26 RF 3
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Conv2d(8, 16, 3),  #input -26  Output - 24 RF 5
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(2, 2) #input -24  Output - 12 RF 10
            )
        self.sequ2 = nn.Sequential(
            nn.Conv2d(16, 32, 3), #input -12 OUtput - 10 RF 12
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.25),
            nn.Conv2d(32, 16, 3),  #input -10  Output - 8 RF 14
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.AdaptiveAvgPool2d((6,6))
            )

        self.fc = nn.Sequential(
            nn.Linear(16*6*6, 10)
            )

    def forward(self, x):
        x = self.sequ1(x)
        x = self.sequ2(x)
        x = x.reshape(-1, 16*6*6)
        x = self.fc(x)
        return F.log_softmax(x)

In [9]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
              ReLU-2            [-1, 8, 26, 26]               0
       BatchNorm2d-3            [-1, 8, 26, 26]              16
            Conv2d-4           [-1, 16, 24, 24]           1,168
              ReLU-5           [-1, 16, 24, 24]               0
       BatchNorm2d-6           [-1, 16, 24, 24]              32
         MaxPool2d-7           [-1, 16, 12, 12]               0
            Conv2d-8           [-1, 32, 10, 10]           4,640
              ReLU-9           [-1, 32, 10, 10]               0
      BatchNorm2d-10           [-1, 32, 10, 10]              64
          Dropout-11           [-1, 32, 10, 10]               0
           Conv2d-12             [-1, 16, 8, 8]           4,624
    

<ipython-input-8-14c2a1af8d27>:33: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [10]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [11]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    # wrong = [0,0,]
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            # print(pred.ne(target.view_as(pred)))
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [15]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.008, momentum=0.9)

for epoch in range(1, 21):
    print("Epoch: ",epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Epoch:  1


  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-8-14c2a1af8d27>:33: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.03100750781595707 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 30.79it/s]



Test set: Average loss: 0.0500, Accuracy: 9849/10000 (98.4900%)

Epoch:  2


loss=0.006264335010200739 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 29.35it/s]



Test set: Average loss: 0.0449, Accuracy: 9870/10000 (98.7000%)

Epoch:  3


...


Test set: Average loss: 0.0206, Accuracy: 9928/10000 (99.2800%)

Epoch:  19


loss=0.0018511704402044415 batch_id=468: 100%|██████████| 469/469 [00:14<00:00, 31.30it/s]



Test set: Average loss: 0.0207, Accuracy: 9932/10000 (99.3200%)

Epoch:  20


loss=0.0015828651376068592 batch_id=468: 100%|██████████| 469/469 [00:14<00:00, 31.51it/s]



Test set: Average loss: 0.0208, Accuracy: 9936/10000 (99.3600%)

